# Modeling prices

##  What are the top criteria linked to pricing your property ? (using ML)

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline

In [155]:
#Loading the data
# Picked up from http://insideairbnb.com/get-the-data/

#df = pd.read_csv('Florence/listings_Sample.csv')
df = pd.read_csv('Florence/listings_Full.csv')



In [156]:
#Our target being the prices we would use the work previously done in Prices.ipynb

#We remove the dollar sign as first character, remove thousands comma separator and convert to float.

df['price_clean'] = df['price'].str[1:].str.replace(",","").astype(float)

# We now need to clean outliers
# In this case we decided to removea the values above 90th percentile and below 10th percentile

df = df[df['price_clean'] <= df['price_clean'].quantile(.9)]
df = df[df['price_clean'] >= df['price_clean'].quantile(.1)]

In [158]:
#Getting last & first review cleaned
from datetime import datetime

df['last_review_cleaned'] = datetime.strptime('1900-01-01 00:00:00','%Y-%m-%d %H:%M:%S')

for ind in df.index:
    if not pd.isna(df['last_review'][ind]):
        df['last_review_cleaned'][ind] = datetime.strptime(str(df['last_review'][ind]),'%Y-%m-%d')
    else :
        df['last_review_cleaned'][ind] = datetime.strptime('1900-01-01 00:00:00','%Y-%m-%d %H:%M:%S')




df['first_review_cleaned'] = datetime.strptime('1900-01-01 00:00:00','%Y-%m-%d %H:%M:%S')

for ind in df.index:
    if not pd.isna(df['first_review'][ind]):
        df['first_review_cleaned'][ind] = datetime.strptime(str(df['first_review'][ind]),'%Y-%m-%d')
    else :
        df['first_review_cleaned'][ind] = datetime.strptime('1900-01-01 00:00:00','%Y-%m-%d %H:%M:%S')


<ipython-input-158-91619fb5d2a1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_review_cleaned'][ind] = datetime.strptime(str(df['last_review'][ind]),'%Y-%m-%d')
<ipython-input-158-91619fb5d2a1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_review_cleaned'][ind] = datetime.strptime('1900-01-01 00:00:00','%Y-%m-%d %H:%M:%S')
<ipython-input-158-91619fb5d2a1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fi

In [159]:
#based on research it looks like only a few amenities are highly searched for by guests: https://www.airbnb.com/resources/hosting-homes/a/the-amenities-guests-want-25

top_amenities_list =['pool','wifi','kitchen','parking','jacuzzi','washer','dryer','airconditioning','heating','petsallowed','selfcheck-in','dedicatedworkspace']

for ind in df.index:

    new_list = (df['amenities'][ind][1:][:-1]).split(",")
    for amenity in new_list:
        amenity = (amenity.replace('"','').lower())
        amenity = amenity.replace(" ","")
        if amenity in top_amenities_list:
            df.at[ind,('has_'+amenity)] = True

In [160]:
#Looking at license

df['has_license'] = False

df['has_license'][df['license'].isna() == False] = True


<ipython-input-160-20038f8fe470>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['has_license'][df['license'].isna() == False] = True


In [190]:
#Looking at host verification


df['has_host_verification'] = np.where(['host_verifications'] == '[]', False, True)

#df[['has_host_verification','host_verifications']]

In [191]:
#Features configuration

features_reviews=[
      'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review_cleaned','last_review_cleaned',
       'review_scores_rating', 'review_scores_accuracy',
      'review_scores_cleanliness', 'review_scores_checkin',
      'review_scores_communication', 'review_scores_location',
      'review_scores_value']
# 0.04779047735933717

features_amenities =['has_pool','has_wifi','has_kitchen','has_parking','has_jacuzzi',
     'has_washer','has_dryer','has_airconditioning','has_heating',
     'has_petsallowed','has_selfcheck-in','has_dedicatedworkspace']

features = ['host_response_time',
       'host_is_superhost', 
       'host_listings_count',
       'host_total_listings_count','has_host_verification'
       'host_has_profile_pic', 'host_identity_verified', 
       'neighbourhood_cleansed', 'room_type', 'accommodates','bedrooms', 'beds', 
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
       'first_review_cleaned','last_review_cleaned',
       'has_pool','has_wifi','has_kitchen','has_parking','has_jacuzzi',
       'has_washer','has_dryer','has_airconditioning','has_heating',
       'has_petsallowed','has_selfcheck-in','has_dedicatedworkspace',
       'has_license']


In [192]:
#Data Preparation

def clean_data(df,features):
    '''
    INPUT
    df - pandas dataframe
    features - list of features to be considered 
    
    OUTPUT
    X - A matrix holding all of the variables you want to consider when predicting the response
    y - the corresponding response vector
    
    This function cleans df using the following steps to produce X and y:
    1. Drop all the rows with no price
    2. Create X as all the columns that are not the price column
    3. Create y as the price column
    4. Filter on selected features to be in X.
    5.a Transform datetime variable to numerical value
    5.b For each numeric variable in X, fill the column with the mean value of the column.
    6. Create dummy columns for all the categorical variables in X, drop the original columns
    '''
    # Drop rows with missing price values
    df = df.dropna(subset=['price_clean'], axis=0)
    y = df['price_clean']
    
    # Filter on list of features
    features = features
    df = df[df.columns.intersection(features)]

    # Handle datetime variable, to convert to numerical values
    date_vars = df.select_dtypes(include=['datetime64']).columns
    for col in date_vars:
        df[col]=df[col].map(datetime.toordinal)

    # Fill numeric columns with the mean
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    for col in num_vars:
        df[col].fillna((df[col].mean()), inplace=True)
        
    # Dummy the categorical variables
    cat_vars = df.select_dtypes(include=['object']).copy().columns
    for var in  cat_vars:
        # for each cat add dummy var, drop original column
        df = pd.concat([df.drop(var, axis=1), pd.get_dummies(df[var], prefix=var, prefix_sep='_', drop_first=True)], axis=1)
    
    X = df
    return X, y
    
#Use the function to create X and y
X, y = clean_data(df, features) 

In [193]:
def clean_fit_linear_mod(X,y, test_size=.3, rand_state=42):
    '''
    INPUT:
    X - a dataframe holding all the features of interest
    y - - a dataframe holding the column target
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    
    Your function should:
    6. Create training and test sets of data
    7. Instantiate a LinearRegression model with normalized data
    8. Fit your model to the training data
    9. Predict the response for the training data and the test data
    10. Obtain an rsquared value for both the training and test data
    '''
    
    # Create training and test sets of data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=rand_state)
    
    # Instantiate a LinearRegression model with normalized data
    lm_model = LinearRegression(normalize=True) # Instantiate
    
    # Fit your model to the training data
    lm_model.fit(X_train, y_train) #Fit
    
    # Predict the response for the training data and the test data
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)
    
    # Obtain an rsquared value for both the training and test data
    test_score = r2_score(y_test, y_test_preds)
    train_score =r2_score(y_train, y_train_preds )
    
    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test



In [194]:

#Training the model & getting scores
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = clean_fit_linear_mod(X,y, test_size=.3, rand_state=42)

print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))


The rsquared on the training data was 0.3644944815373611.  The rsquared on the test data was 0.3731204571263109.


In [195]:
def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df


In [199]:
#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df.head(20)
coef_df.style.bar(subset=['coefs'], align='mid', color=['#d65f5f', '#5fba7d'])


,est_int,coefs,abs_coefs
47,has_availability_t,-62.205505,62.205505
46,room_type_Shared room,-39.713419,39.713419
25,review_scores_location,21.836343,21.836343
40,neighbourhood_cleansed_Centro Storico,20.609088,20.609088
3,bedrooms,16.464596,16.464596
41,neighbourhood_cleansed_Gavinana Galluzzo,12.046478,12.046478
22,review_scores_cleanliness,11.509169,11.509169
42,neighbourhood_cleansed_Isolotto Legnaia,-9.782779,9.782779
2,accommodates,9.771135,9.771135
43,neighbourhood_cleansed_Rifredi,-8.773816,8.773816
